
<font color = green>

#  Home task: Support Vector Machine
</font>




In [1]:
import pandas as pd 
import numpy as np

In [2]:
import os
cwd = os.getcwd()                 # Current working directory
path = os.path.join(cwd, 'data')  # Data directory

***
<font color = green>

##  Sklearn SVM for Iris dataset
</font>



In [3]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [4]:
# Load Iris dataset
X, y = load_iris(return_X_y=True)

# Split data into training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

# Normalize feature values
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [5]:
# YOUR_CODE.  Try linear, rbf and poly kernels 
# START_CODE
from sklearn.svm import SVC

# Use SVM classifier with linear kernel
clf_svm = SVC(C=0.1, kernel='linear').fit(X_train_scaled, y_train)
print('SVM classifier (linear kernel):')
print('train accuracy = {:.3%}'.format(clf_svm.score(X_train_scaled, y_train)))
print('test accuracy = {:.3%}'.format(clf_svm.score(X_test_scaled, y_test)), end='\n\n')

# Use SVM classifier with polynomial kernel
clf_svm = SVC(C=0.1, kernel='poly', degree=3).fit(X_train_scaled, y_train)
print('SVM classifier (polynomial kernel, degree = 3):')
print('train accuracy = {:.3%}'.format(clf_svm.score(X_train_scaled, y_train)))
print('test accuracy = {:.3%}'.format(clf_svm.score(X_test_scaled, y_test)), end='\n\n')

# Use SVM classifier with RBF kernel
clf_svm = SVC(C=0.1, kernel='rbf', gamma=0.5).fit(X_train_scaled, y_train)
print('SVM classifier (RBF kernel, gamma = 0.5):')
print('train accuracy = {:.3%}'.format(clf_svm.score(X_train_scaled, y_train)))
print('test accuracy = {:.3%}'.format(clf_svm.score(X_test_scaled, y_test)))
# END_CODE 

SVM classifier (linear kernel):
train accuracy = 96.429%
test accuracy = 97.368%

SVM classifier (polynomial kernel, degree = 3):
train accuracy = 86.607%
test accuracy = 84.211%

SVM classifier (RBF kernel, gamma = 0.5):
train accuracy = 93.750%
test accuracy = 94.737%


***
<font color = green>

##  Spam/Non-spam classification
</font>




<font color = green>

###  Problem statement
</font>

Spam filter classifies emails into spam and non-spam email. 
For this task use SVMs to build your spam filter.
This is binary classification  - whether a given email, x, is spam (y = 1) or non-spam (y = 0).
In particular, you need to convert each email into a feature vector x. 
The dataset for this task is based on a a subset of the SpamAssassin Public Corpus.
For the purpose of this exercise, it will use the body of the email (excluding the email headers)


<font color = green>

###  Preprocessing mail
</font>

Before starting on a machine learning task, it is usually insightful to take a look at examples from the dataset. Below is a sample email that contains a URL, an email address (at the end), numbers, and dollar amounts. While many emails would contain similar types of entities (e.g., numbers, other URLs, or other email addresses), the specific entities (e.g., the specific URL or specific dollar amount) will be different in almost every email. Therefore, one method often employed in processing emails is to “normalize” these values, so that all URLs are treated the same, all numbers are treated the same, etc. For example, replace each URL in the email with the unique string “http_addr” to indicate that a URL was present.
This lets the spam classifier to make a classification decision based on whether any URL was present, rather than whether a specific URL was present. This typically improves the performance of a spam classifier, since spammers often randomize the URLs, and thus the odds of seeing any particular URL again in a new piece of spam is very small.


<font color = green>

### Reveiw sample mail 
</font>



In [6]:
def get_sample(filename):
    with open(filename, 'r') as f:
        content = f.read()
    return content
    
file = os.path.join(path , 'emailSample1.txt')
content = get_sample(file)
print(content)
content

> Anyone knows how much it costs to host a web portal ?
>
Well, it depends on how many visitors you're expecting.
This can be anywhere from less than 10 bucks a month to a couple of $100. 
You should checkout http://www.rackspace.com/ or perhaps Amazon EC2 
if youre running something big..

To unsubscribe yourself from this mailing list, send an email to:
groupname-unsubscribe@egroups.com




"> Anyone knows how much it costs to host a web portal ?\n>\nWell, it depends on how many visitors you're expecting.\nThis can be anywhere from less than 10 bucks a month to a couple of $100. \nYou should checkout http://www.rackspace.com/ or perhaps Amazon EC2 \nif youre running something big..\n\nTo unsubscribe yourself from this mailing list, send an email to:\ngroupname-unsubscribe@egroups.com\n\n"

<font color = green>

### Preprocessing and normalization steps
</font>

Preprocessing and normalization includes the following steps:
<ul>
<li> <b>Lower-casing</b>: The entire email is converted into lower case, so that capitalization is ignored (e.g., IndIcaTE is treated the same as Indicate).
    
<li> <b>Stripping HTML</b>: All HTML tags are removed from the emails. Many emails often come with HTML formatting; we remove all the HTML tags, so that only the content remains.
<li> <b>Normalizing URLs</b>: All URLs are replaced with the text “httpaddr”.
<li> <b>Normalizing Email Addresses</b>: All email addresses are replaced
with the text “emailaddr”.
<li> <b>Normalizing Numbers</b>: All numbers are replaced with the text
“number”.
<li> <b>Normalizing Dollars</b>: All dollar signs ($) are replaced with the text
“dollar”.
<li> <b>Word Stemming</b>: Words are reduced to their stemmed form. For example, “discount”, “discounts”, “discounted” and “discounting” are all replaced with “discount”. Sometimes, the Stemmer actually strips on additional characters from the end, so “include”, “includes”, “included”, and “including” are all replaced with “includ”.
<li> <b>Removal of non-words</b>: Non-words and punctuation have been removed. All white spaces (tabs, newlines, spaces) have all been trimmed to a single space character.
</ul>

<font color = green>

### Word tokenization
</font>


In [7]:
import re

In [8]:
def word_tokenize(content):
    '''
    content: str - body of mail
    return: list of tokens (str) e.g. ['>', 'Anyone', 'knows', 'how', 'much', 'it', 'costs', 'to', 'host', 'a']
    '''
    # YOUR_CODE.  Split the content to tokens. You may need re.split()
    # START_CODE
    tokens = re.split('\s', content)
    # END_CODE
    
    return np.array(tokens)

<font color = blue >

### Check result

</font>


In [9]:
tokens = word_tokenize('''> Anyone knows how much it costs to host a web portal ?\n>\nWell, it depends on how many visitors you're expecting.\nThis can be anywhere from less than 10 bucks a month to a couple of $100. \nYou should checkout http://www.rackspace.com/ or perhaps Amazon EC2 \nif youre running something big..\n\nTo unsubscribe yourself from this mailing list, send an email to:\ngroupname-unsubscribe@egroups.com\n\n''')
tokens

array(['>', 'Anyone', 'knows', 'how', 'much', 'it', 'costs', 'to', 'host',
       'a', 'web', 'portal', '?', '>', 'Well,', 'it', 'depends', 'on',
       'how', 'many', 'visitors', "you're", 'expecting.', 'This', 'can',
       'be', 'anywhere', 'from', 'less', 'than', '10', 'bucks', 'a',
       'month', 'to', 'a', 'couple', 'of', '$100.', '', 'You', 'should',
       'checkout', 'http://www.rackspace.com/', 'or', 'perhaps', 'Amazon',
       'EC2', '', 'if', 'youre', 'running', 'something', 'big..', '',
       'To', 'unsubscribe', 'yourself', 'from', 'this', 'mailing',
       'list,', 'send', 'an', 'email', 'to:',
       'groupname-unsubscribe@egroups.com', '', ''], dtype='<U33')

<font color = blue >

### Expected output

</font>

`array(['>', 'Anyone', 'knows', 'how', 'much', 'it', 'costs', 'to', 'host',
       'a', 'web', 'portal', '?', '>', 'Well', 'it', 'depends', 'on',
       'how', 'many', 'visitors', "you're", 'expecting.', 'This', 'can',
       'be', 'anywhere', 'from', 'less', 'than', '10', 'bucks', 'a',
       'month', 'to', 'a', 'couple', 'of', '$100.', '', 'You', 'should',
       'checkout', 'http://www.rackspace.com/', 'or', 'perhaps', 'Amazon',
       'EC2', '', 'if', 'youre', 'running', 'something', 'big..', '',
       'To', 'unsubscribe', 'yourself', 'from', 'this', 'mailing', 'list',
       'send', 'an', 'email', 'to:', 'groupname-unsubscribe@egroups.com',
       '', ''], dtype='<U33')`

<font color = green>

### Lower case 
</font>


In [10]:
def lower_case(tokens):
    '''
    tokens: ndarray of str
    return: ndarray of tokens in lower case (str)
    '''
    # YOUR_CODE.  Make all tokens in lower case
    # START_CODE
    tokens = np.vectorize(str.lower)(tokens)
    # END_CODE
   
    return tokens

<font color = blue >

### Check result

</font>


In [11]:
tokens = lower_case(tokens)
tokens

array(['>', 'anyone', 'knows', 'how', 'much', 'it', 'costs', 'to', 'host',
       'a', 'web', 'portal', '?', '>', 'well,', 'it', 'depends', 'on',
       'how', 'many', 'visitors', "you're", 'expecting.', 'this', 'can',
       'be', 'anywhere', 'from', 'less', 'than', '10', 'bucks', 'a',
       'month', 'to', 'a', 'couple', 'of', '$100.', '', 'you', 'should',
       'checkout', 'http://www.rackspace.com/', 'or', 'perhaps', 'amazon',
       'ec2', '', 'if', 'youre', 'running', 'something', 'big..', '',
       'to', 'unsubscribe', 'yourself', 'from', 'this', 'mailing',
       'list,', 'send', 'an', 'email', 'to:',
       'groupname-unsubscribe@egroups.com', '', ''], dtype='<U33')

<font color = blue >

### Expected output

</font>

`array(['>', 'anyone', 'knows', 'how', 'much', 'it', 'costs', 'to', 'host',
       'a', 'web', 'portal', '?', '>', 'well', 'it', 'depends', 'on',
       'how', 'many', 'visitors', "you're", 'expecting.', 'this', 'can',
       'be', 'anywhere', 'from', 'less', 'than', '10', 'bucks', 'a',
       'month', 'to', 'a', 'couple', 'of', '$100.', '', 'you', 'should',
       'checkout', 'http://www.rackspace.com/', 'or', 'perhaps', 'amazon',
       'ec2', '', 'if', 'youre', 'running', 'something', 'big..', '',
       'to', 'unsubscribe', 'yourself', 'from', 'this', 'mailing', 'list',
       'send', 'an', 'email', 'to:', 'groupname-unsubscribe@egroups.com',
       '', ''], dtype='<U33')`

<font color = green>

### Normalize
</font>


In [12]:
def normalize_tokens(tokens):
    '''
    tokens: ndarray of str
    return: ndarray of tokens replaced with corresponding unified words
    '''
    # YOUR_CODE.  
        # remove html and other tags
        # mark all numbers as 'number'
        # mark all urls as 'httpaddr'
        # mark all emails as 'emailaddr'
        # replace $ as 'dollar'
        # get rid of any punctuation
        # remove any non alphanumeric characters
    #  You may  need re.sub()
    # START_CODE

    # Replace urls with 'httpaddr'
    tokens = np.vectorize(lambda x: re.sub(r'https?://(www.)?[\w\d!"#$%&\'()*+-,./:;<=>?@\^_`{|}~-]+', 'httpaddr', x))(tokens)

    # Replace emails with 'emailaddr'
    email_pattern = (
        r'[\w\d!"#$%&\'()*+-,/:;<=>?\^_`{|}~-]+\.*[\w\d!"#$%&\'()*+-,/:;<=>?\^_`{|}~-]+'  # Local part
        r'@'                                                                              # at sign
        r'[\w\d!"#$%&\'()*+,./:;<=>?\^_`{|}~-]+\-*[\w\d!"#$%&\'()*+,./:;<=>?\^_`{|}~-]+'  # Domain part
    )
    tokens = np.vectorize(lambda x: re.sub(email_pattern, 'emailaddr', x))(tokens)

    # Replace dollar sign '$' with 'dollar'
    tokens = np.vectorize(lambda x: re.sub(r'\$', 'dollar', x))(tokens)

    # Remove html tags, punctuation and non alphanumeric characters
    tokens = np.vectorize(lambda x: re.sub(r'(<.+?>)|[!"#$%&\'()*+-,./:;<=>?@\^_`{|}~-]|\W', '', x))(tokens)

    # Replace numbers with 'number'
    tokens = np.vectorize(lambda x: re.sub(r'\d+', 'number', x))(tokens)
    # END_CODE

    return tokens

<font color = blue >

### Check result

</font>


In [13]:
tokens = normalize_tokens(tokens)
tokens

array(['', 'anyone', 'knows', 'how', 'much', 'it', 'costs', 'to', 'host',
       'a', 'web', 'portal', '', '', 'well', 'it', 'depends', 'on', 'how',
       'many', 'visitors', 'youre', 'expecting', 'this', 'can', 'be',
       'anywhere', 'from', 'less', 'than', 'number', 'bucks', 'a',
       'month', 'to', 'a', 'couple', 'of', 'dollarnumber', '', 'you',
       'should', 'checkout', 'httpaddr', 'or', 'perhaps', 'amazon',
       'ecnumber', '', 'if', 'youre', 'running', 'something', 'big', '',
       'to', 'unsubscribe', 'yourself', 'from', 'this', 'mailing', 'list',
       'send', 'an', 'email', 'to', 'emailaddr', '', ''], dtype='<U12')

<font color = blue >

### Expected output

</font>

`array(['', 'anyone', 'knows', 'how', 'much', 'it', 'costs', 'to', 'host',
       'a', 'web', 'portal', '', '', 'well', 'it', 'depends', 'on', 'how',
       'many', 'visitors', 'youre', 'expecting', 'this', 'can', 'be',
       'anywhere', 'from', 'less', 'than', 'number', 'bucks', 'a',
       'month', 'to', 'a', 'couple', 'of', 'dollarnumber', '', 'you',
       'should', 'checkout', 'httpaddr', 'or', 'perhaps', 'amazon',
       'ecnumber', '', 'if', 'youre', 'running', 'something', 'big', '',
       'to', 'unsubscribe', 'yourself', 'from', 'this', 'mailing', 'list',
       'send', 'an', 'email', 'to', 'emailaddr', '', ''], dtype='<U12')`

<font color = green>

### Remove zero length tokens
</font>


In [14]:
def filter_short_tokens(tokens):
    '''
    tokens: ndarray of str
    return: ndarray of filtered tokens (str)
    '''
    original_tokens_len = len(tokens)

    # YOUR_CODE. Keep only tokens that length > 0  
    # START_CODE
    tokens = tokens[np.vectorize(lambda x: len(x) > 0)(tokens)]
    # END_CODE

    print('Original len = {}\nRemaining len = {}'.format(original_tokens_len, len(tokens)))    

    return tokens

<font color = blue >

### Check result

</font>


In [15]:
tokens = filter_short_tokens(tokens)
tokens

Original len = 69
Remaining len = 61


array(['anyone', 'knows', 'how', 'much', 'it', 'costs', 'to', 'host', 'a',
       'web', 'portal', 'well', 'it', 'depends', 'on', 'how', 'many',
       'visitors', 'youre', 'expecting', 'this', 'can', 'be', 'anywhere',
       'from', 'less', 'than', 'number', 'bucks', 'a', 'month', 'to', 'a',
       'couple', 'of', 'dollarnumber', 'you', 'should', 'checkout',
       'httpaddr', 'or', 'perhaps', 'amazon', 'ecnumber', 'if', 'youre',
       'running', 'something', 'big', 'to', 'unsubscribe', 'yourself',
       'from', 'this', 'mailing', 'list', 'send', 'an', 'email', 'to',
       'emailaddr'], dtype='<U12')

<font color = blue >

### Expected output

</font>

`Original len= 69
Remaining len= 61
array(['anyone', 'knows', 'how', 'much', 'it', 'costs', 'to', 'host', 'a',
       'web', 'portal', 'well', 'it', 'depends', 'on', 'how', 'many',
       'visitors', 'youre', 'expecting', 'this', 'can', 'be', 'anywhere',
       'from', 'less', 'than', 'number', 'bucks', 'a', 'month', 'to', 'a',
       'couple', 'of', 'dollarnumber', 'you', 'should', 'checkout',
       'httpaddr', 'or', 'perhaps', 'amazon', 'ecnumber', 'if', 'youre',
       'running', 'something', 'big', 'to', 'unsubscribe', 'yourself',
       'from', 'this', 'mailing', 'list', 'send', 'an', 'email', 'to',
       'emailaddr'], dtype='<U12')`

<font color = green>

### Stem tokens
</font>


In [16]:
from nltk.stem import PorterStemmer

In [17]:
def stem_tokens(tokens):
    '''
    tokens: ndarry of str
    return: ndarry of stemmed tokens e.g. array(['anyon', 'know', 'how', 'much', 'it', 'cost', 'to', 'host', 'a',
       'web', 'portal', 'well', 'it', 'depend', 'on', 'how', 'mani']...
    '''
    # YOUR_CODE. replace the tokens by stemmed form. You may need PorterStemmer.stem() 
    # START_CODE
    stemmer = PorterStemmer()
    tokens = np.vectorize(stemmer.stem)(tokens)
    # END_CODE
   
    return tokens

<font color = blue >

### Check result

</font>


In [18]:
tokens = stem_tokens(tokens)
tokens

array(['anyon', 'know', 'how', 'much', 'it', 'cost', 'to', 'host', 'a',
       'web', 'portal', 'well', 'it', 'depend', 'on', 'how', 'mani',
       'visitor', 'your', 'expect', 'thi', 'can', 'be', 'anywher', 'from',
       'less', 'than', 'number', 'buck', 'a', 'month', 'to', 'a', 'coupl',
       'of', 'dollarnumb', 'you', 'should', 'checkout', 'httpaddr', 'or',
       'perhap', 'amazon', 'ecnumb', 'if', 'your', 'run', 'someth', 'big',
       'to', 'unsubscrib', 'yourself', 'from', 'thi', 'mail', 'list',
       'send', 'an', 'email', 'to', 'emailaddr'], dtype='<U10')

<font color = blue >

### Expected output

</font>

`array(['anyon', 'know', 'how', 'much', 'it', 'cost', 'to', 'host', 'a',
       'web', 'portal', 'well', 'it', 'depend', 'on', 'how', 'mani',
       'visitor', 'your', 'expect', 'thi', 'can', 'be', 'anywher', 'from',
       'less', 'than', 'number', 'buck', 'a', 'month', 'to', 'a', 'coupl',
       'of', 'dollarnumb', 'you', 'should', 'checkout', 'httpaddr', 'or',
       'perhap', 'amazon', 'ecnumb', 'if', 'your', 'run', 'someth', 'big',
       'to', 'unsubscrib', 'yourself', 'from', 'thi', 'mail', 'list',
       'send', 'an', 'email', 'to', 'emailaddr'], dtype='<U10')`

<font color = green>

### Vocabulary
</font>

Ususally vocabulary is built from the top most frequent tokens within all available training set.
For this task it is already prepared and provided in the `vocab.txt`

In [19]:
def get_vocabulary(filename):
    '''
    fn: str - full path to file 
    return: ndarray of str e.g. array(['aa', 'ab', 'abil', ..., 'zdnet', 'zero', 'zip'], dtype=object)
    '''
    vocab_list = pd.read_table(filename, header=None)
    vocab = np.array(vocab_list)[:, 1]  # First columns is index, select only words column  
    print('len(vocab) = {}'.format(len(vocab)))
    return vocab

file = os.path.join(path, 'vocab.txt')
vocab = get_vocabulary(file)
vocab

len(vocab) = 1899


array(['aa', 'ab', 'abil', ..., 'zdnet', 'zero', 'zip'], dtype=object)

<font color = green>

### Feature reresentation
</font>

Every word in vocabulary is the feature of the mail - 1 is the word in in the mail and 0 otherwise. 
Thus every mail can be represented as the binary array of fixed length - number of words in vocabulary.

In [20]:
def represent_features(tokens, vocab):
    '''
    tokens: ndarray of str
    tokens: ndarray of str
    return: ndarray of binary values 1 if word from vocabulary is in mail 0 otherwise
    '''
    # YOUR_CODE. Compute the array with 1/0 corresponding to is word from vocabulary in mail 
    # START_CODE
    tokens_represented = np.vectorize(lambda x: 1 if x in tokens else 0)(vocab)
    # END_CODE

    print('{} word(s) from vocab are in the tokens.'.format(np.sum(tokens_represented)))
    return tokens_represented


<font color = blue >

### Check result

</font>


In [21]:
tokens_represented = represent_features(tokens, vocab)
tokens_represented

44 word(s) from vocab are in the tokens.


array([0, 0, 0, ..., 0, 0, 0])

<font color = blue >

### Expected output

</font>

`44 word(s) from vocab are in the tokens.
array([0, 0, 0, ..., 0, 0, 0])`

<font color = green>

### Composing all steps of preprocessing 
</font>

Every word in vocabulary is the feature of the mail - 1 is the word in in the mail and 0 otherwise. 
Thus every mail can be represented as the binary array of fixed length - number of words in vocabulary.

In [22]:
def preprocess(content, vocab):
    '''
    content: str - body of mail 
    vocab: ndarray of str - list of considered words 
    '''
    # YOUR_CODE. Compute the array with 1/0 corresponding to is word from vocabulary in mail 
    # START_CODE 

    # Tokenize content    
    tokens  = word_tokenize(content)
    
    # Make lower case
    tokens = lower_case(tokens)

    # Normalize tokens
    tokens = normalize_tokens(tokens)

    # Remove zero words
    tokens = filter_short_tokens(tokens)
    
    # Stem words
    tokens = stem_tokens(tokens)
    
    # Convert to binary array of features
    tokens_represented = represent_features(tokens, vocab)
    # END_CODE     
    
    return tokens_represented

<font color = blue >

### Check result

</font>


In [23]:
preprocess(content, vocab)

Original len = 69
Remaining len = 61
44 word(s) from vocab are in the tokens.


array([0, 0, 0, ..., 0, 0, 0])

<font color = blue >

### Expected output

</font>

`Original len= 69
Remaining len= 61
44 word(s) from vocab are in the tokens.
array([0, 0, 0, ..., 0, 0, 0])`

<font color = green>

### Training and test sets
</font>

All mails for training are preprocessed the same way as performed for sample above. 
The training set (array of feature representation) is provided in the `spamTrain.mat`
The corresponding test set is provided in the `spamTest.mat`

In [24]:
from scipy.io import loadmat

In [25]:
# Load training data
file = os.path.join(path , 'spamTrain.mat')
mat = loadmat(file)
X_train = mat['X']
y_train = mat['y'].ravel()

print('X_train.shape =', X_train.shape)
print('y_train.shape =', y_train.shape)

# Load testing data
file = os.path.join(path, 'spamTest.mat')
mat = loadmat(file)
X_test = mat['Xtest']
y_test = mat['ytest'].ravel() 

print('X_test.shape =', X_test.shape)
print('y_test.shape =', y_test.shape)

index = 0
print('Sample with index = {}: \n{}'.format(index, X_train[index]))

X_train.shape = (4000, 1899)
y_train.shape = (4000,)
X_test.shape = (1000, 1899)
y_test.shape = (1000,)
Sample with index = 0: 
[0 0 0 ... 0 0 0]


<font color = green>

### Training  the model
</font>


In [26]:
from sklearn.svm import LinearSVC

In [27]:
clf = LinearSVC(C=0.1)
clf.fit(X_train, y_train)
print('Score train = {}'.format(clf.score(X_train, y_train)))
print('Score test = {}'.format(clf.score(X_test, y_test)))

Score train = 0.99975
Score test = 0.992


<font color = green>

### Determining most spam contributors
</font>


In [28]:
# YOUR_CODE. Compute top 20 largest coefficients and return the corresponding 20 words from vocabulary
# START_CODE

# Get coefficients of SVM classifier
coefs = np.squeeze(clf.coef_)

# Sort coefficients in descending order
top_spam = np.flip(np.argsort(coefs))

top_spam_contributors = vocab[top_spam[:20]]
# END_CODE

<font color = blue >

### Check result

</font>


In [29]:
print(top_spam_contributors)

['our' 'remov' 'click' 'basenumb' 'guarante' 'visit' 'bodi' 'will'
 'numberb' 'price' 'dollar' 'nbsp' 'below' 'lo' 'most' 'send' 'dollarnumb'
 'credit' 'wi' 'hour']


<font color = blue >

### Expected output

</font>

`['our' 'remov' 'click' 'basenumb' 'guarante' 'visit' 'bodi' 'will'
 'numberb' 'price' 'dollar' 'nbsp' 'below' 'lo' 'most' 'send' 'dollarnumb'
 'credit' 'wi' 'hour']`

<font color = green>

### Use model for prediction 
</font>


In [30]:
for sample_file in ['emailSample1.txt', 'emailSample2.txt', 'spamSample1.txt', 'spamSample2.txt']:
    file = os.path.join(path, sample_file)    
    content = get_sample(file)
    
    # YOUR_CODE.  Preprocess the sample and get prediction 0 or 1 (1 is spam)
    # START_CODE
    prediction = clf.predict(preprocess(content, vocab).reshape(1, -1))[0]
    # END_CODE
    
    print('{} is {}\n'.format(sample_file, ('Not Spam', 'Spam')[prediction]))

print('Latter sample:\n{1}\n{0}\n{1}'.format(content, '='*50))

Original len = 69
Remaining len = 61
44 word(s) from vocab are in the tokens.
emailSample1.txt is Not Spam

Original len = 247
Remaining len = 222
122 word(s) from vocab are in the tokens.
emailSample2.txt is Not Spam

Original len = 141
Remaining len = 97
46 word(s) from vocab are in the tokens.
spamSample1.txt is Spam

Original len = 39
Remaining len = 31
18 word(s) from vocab are in the tokens.
spamSample2.txt is Spam

Latter sample:
Best Buy Viagra Generic Online

Viagra 100mg x 60 Pills $125, Free Pills & Reorder Discount, Top Selling 100% Quality & Satisfaction guaranteed!

We accept VISA, Master & E-Check Payments, 90000+ Satisfied Customers!
http://medphysitcstech.ru





<font color = blue >

### Expected output

</font>

`Original len= 69
Remaining len= 61
44 word(s) from vocab are in the tokens.
emailSample1.txt is Not Spam`

`Original len= 247
Remaining len= 222
122 word(s) from vocab are in the tokens.
emailSample2.txt is Not Spam`

`Original len= 141
Remaining len= 97
46 word(s) from vocab are in the tokens.
spamSample1.txt is Spam`

`Original len= 39
Remaining len= 31
18 word(s) from vocab are in the tokens.
spamSample2.txt is Spam`

`Latter sample:`

`==================================================`

`Best Buy Viagra Generic Online`

`Viagra 100mg x 60 Pills $125, Free Pills & Reorder Discount, Top Selling 100\% Quality & Satisfaction guaranteed!`

`We accept VISA, Master & E-Check Payments, 90000+ Satisfied Customers!
http://medphysitcstech.ru`


`==================================================`

<font color = green>

# Congratulation!
</font>

Now you know how to build Spam/Non-Spam classifier

